In [ ]:
import os
from os import listdir
from os.path import isfile, join
import pickle
import pandas as pd
import plotly.express as px
import plotly.figure_factory
import plotly.figure_factory as ff

In [ ]:
list_dict = []
mypath = 'artifacts/best_movielens'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
for file in onlyfiles:
    with open(os.path.join(mypath,file), 'rb') as handle:
            train_dict = pickle.load(handle)
            list_dict.append(train_dict)


In [ ]:
# for train_dict in list_dict:
#     train_dict['mse_valid'] = train_dict[ 'val_score_mse_loss_arr'][-1]
#     train_dict['mse_train'] = train_dict[ 'train_score_mse_loss_arr'][-1]
#     train_dict['unfair_score'] = train_dict[ 'train_score_unfair_arr'][-1]


In [ ]:
keys_nmf = {}
for key in list(train_dict['val_dict_unfair']):
    key_list = []
    for train_dict in list_dict:
        if train_dict['model_type'] == 'nmf':
            key_list.append(train_dict['val_dict_unfair'][key])
        key_dict = {key: key_list}
        keys_nmf={**keys_nmf,**key_dict}

keys_gmf = {}
for key in list(train_dict['val_dict_unfair']):
    key_list = []
    for train_dict in list_dict:
        if train_dict['model_type'] == 'gmf':
            key_list.append(train_dict['val_dict_unfair'][key])
        key_dict = {key: key_list}
        keys_gmf={**keys_gmf,**key_dict}


In [ ]:
hp = pd.DataFrame.from_dict(keys,orient='columns')[['val_score','abs_score','under_score','non_parity_score','rmse']]
hp['model'] = hp.index

In [ ]:
def plot_tuning(tuning_data:pd.DataFrame, hyperparam_cols:str ,metric_cols:list ,model_col = 'model_type'):
    hp_results_melt = tuning_data.melt(id_vars=[model_col]+ hyperparam_cols, value_vars=metric_cols)
    for i, hyperparam_col in enumerate(hyperparam_cols):
        hp_results_melt = hp_results_melt.sort_values(by =hyperparam_col)
        hp_results_melt[hyperparam_col] = hp_results_melt[hyperparam_col].astype('str')
        fig = px.box(hp_results_melt, x=hyperparam_col, y='value',  color='model_type', facet_col='variable',facet_col_wrap=2, title= hyperparam_col)
        fig.update_yaxes(matches=None)
        fig.update_yaxes(showticklabels=True)
        fig.show()



In [ ]:
plot_tuning(tuning_data=hp, metric_cols= ['mse_train','mse_valid','unfair_score','fit_time'] , hyperparam_cols=['metric','embed_dim','lr'])

In [ ]:
def plot_metric_comparison(best_models:pd.DataFrame,metric_cols:list,time_col:str= None):
    best_models['sum'] = best_models[metric_cols].abs().sum(axis=1)
    best_models=best_models.sort_values(by='sum')
    best_models['model']=best_models['model'].astype('str')
    fig = px.bar(best_models,x='model', y=metric_cols, text_auto=True, opacity=0.7, title = 'metric comparison')
    fig.show()
    if time_col:

        fig = px.bar(best_models,x='model', y=time_col, text_auto=True, opacity=0.7, title = 'fit time (sec)')
        fig.show()


In [ ]:
plot_metric_comparison(hp,['mse_train','mse_valid','unfair_score'],'fit_time')

In [ ]:
df_gmf = pd.read_csv('res_gmf_30e_val.csv')
df_gmf = df_gmf.rename({'Unnamed: 0':'metric','Unnamed: 6':'rmse' },axis=1)

In [ ]:
metrics = ['val_score','abs_score','over_score','under_score','non_parity_score','rmse']

In [ ]:
for test in df_gmf.test.unique():
    cur_df = df_gmf[df_gmf.test ==test]    
    fig = ff.create_annotated_heatmap(z=cur_df[metrics].round(5).values, x=metrics, y=metrics, colorscale='Geyser', zmin=0, zmax=0.5,
                                      showscale=True,opacity=0.9)  # redgreen platte
    fig.update_layout(width=600, height=350, xaxis_nticks=36, title=test)
    fig['layout']['xaxis']['autorange'] = "reversed"

    fig.show()


In [ ]:
sum_df = df_gmf[df_gmf.test ==test] 
sum_df[metrics] =0
for test in df_gmf.test.unique():
    cur_df = df_gmf[df_gmf.test ==test] 
    sum_df[metrics] =sum_df[metrics] + cur_df[metrics].values
    sum_df[metrics] = sum_df[metrics]/4
    
fig = ff.create_annotated_heatmap(z=sum_df[metrics].round(5).values, x=metrics, y=metrics, colorscale='Geyser', zmin=0, zmax=0.2,
                                  showscale=True,opacity=0.9)  # redgreen platte
fig.update_layout(width=600, height=350, xaxis_nticks=36, title='mean')
fig['layout']['xaxis']['autorange'] = "reversed"

fig.show()


In [ ]:
tune_df = pd.read_csv('tuning_syn.csv')

In [ ]:
tune_df =tune_df.rename({'Unnamed: 0': 'objective','Unnamed: 10':'mse'},axis=1)

In [ ]:
metrics =  ['val_score', 'abs_score', 'over_score', 'under_score', 'non_parity_score', 'mse']
tune_params = ['objective','model','unfairness_reg','lr','test']

In [ ]:
# tune_df_copy = tune_df.copy()
# tune_df_copy['model'] = tune_df_copy.index
# plot_metric_comparison(tune_df_copy,metrics)

In [ ]:
tune_df['lr']=tune_df['lr'].round(1)
tune_df['unfairness_reg']=tune_df['unfairness_reg'].round(1)
tune_df['early_stop']=tune_df_copy['early_stop'].round(6)

tune_df['model_type']= tune_df['model']

In [ ]:
plot_tuning(tuning_data=tune_df, metric_cols= metrics , hyperparam_cols=['unfairness_reg','objective','lr','model_type','early_stop'],model_col ='model')